## 运行环境说明：

In [65]:
import sys
import platform


print("系统版本是：{}".format(' '.join(platform.linux_distribution())))
print("Python 版本是：{}".format(sys.version.split()[0]))

系统版本是：debian 8.7 
Python 版本是：3.5.2


# 2w 作业

* 任务1：贝叶斯公式的运用    
	利用贝叶斯公式，说明为什么 $P(y|w1, w2) ≠ P(y|w1)P(y|w2)$（即使做了独立假设）
    
* 任务2：实现 Naive Bayes 方法    
	请你用 Python 实现 Naive Bayes 方法，并在给定的数据集上验证数据。具体要求如下：    
    在「训练数据」上拟合一个 Naive Bayes 模型。在训练时模型不能「看见」任何测试数据的信息。    
    训练完成后，在测试数据上进行测试。评估标准为你的模型在测试数据上的混淆矩阵（Confusion Matrix）结果。    
    根据混淆矩阵的结果，分析一下你模型的表现。    
	参考概念：混淆矩阵 Simple guide to confusion matrix terminology    
    
* 任务3：实现 Gradient Descent 算法    
	通过梯度下降法，自己实现一种通用的给定数据找到 y = wx + b 中最优的 w 和 b 的程序，并用加噪音数据验证效果。


## 1. 贝叶斯公式的运用

$P(y|w_1) = \frac{P(w_1|y)P(y)}{P(w_1)}$

$P(y|w_2) = \frac{P(w_2|y)P(y)}{p(w_2)}$

$P(y|w_1)P(y|w_2) = \frac{P(w_1|y)P(w_2|y)P(y)^2}{p(w_1)P(w_2)}$

$P(y|w_1,w_2) = \frac{P(w_1,w_2|y)p(y)}{P(w_1,w_2)}$

从上面两个公式可以看出，哪怕做了独立假设，

$P(y|w_1).P(y|w_2) 也是不等于 $P(y|w_1,w_2) 

## 2. 实现 Naive Bayes 方法

### 贝叶斯公式和应用：

贝叶斯的核心是通过先验概率和逆条件概率从而求出条件概率：

* 正面和负面分别占比，是其中之一的先验概率
* 已知文本的情感，各个词所占的概率是逆条件概率

### 作业思路和解决步骤：

1. 分词，并剔除停止词(影响情感)
2. 统计各个词在正负面所占的概率，并计算他们的联合概率(考虑独立性)
3. 生成模型(还要细化。。。)


### Refrences:

* [Bayesian Classification withInsect examples](http://www.cs.ucr.edu/~eamonn/CE/Bayesian%20Classification%20withInsect_examples.pdf) 这个 Slide 超赞

### 2.1 读取并分词

In [1]:
import os
import string
import re
import pyprind
import multiprocessing
import jieba
import numpy as np
import pandas as pd

from collections import Counter

首先用 linux 命令简单查看下数据

In [2]:
!head -n 5 data/pos_train.txt | nl

     1	装了xp系统后，没有出现网友说的驱动不好装的情况
     2	总的来说,比较干净,而且地理位置很好,市区繁华地段.进出方便.
     3	2、散热很好，这个不用解释了
     4	温度控制的非常好，噪音也不大，
     5	早上6点多有"按摩"电话过来，^_^；不想被打扰的话拔掉电话插头吧


发现数据每一行都是针对不同商品且不一样的评论，所以一行其实就是一个情绪。

参考 [rasbt/python-machine-learning-book](https://github.com/rasbt/python-machine-learning-book) 把每行读取出来，且生成标签

In [3]:
basepath = './data'

labels = {'pos_train': 1, 'neg_train':0, 'neg_test': '0?', 'pos_test': '1?'}
pbar = pyprind.ProgBar(36000)  # 迭代次数
df = pd.DataFrame()
for i in labels:
    path = os.path.join(basepath, i + '.txt')
    with open(path) as f:
        lines = (line.strip() for line in f.readlines())
    for line in lines:
        df = df.append([[line, labels[i]]], ignore_index=True)
        pbar.update()
df.columns = ['review', 'sentiment']

0%                          100%
[############################# ] | ETA: 00:00:02

In [4]:
df.head()

,review,sentiment
0,光驱不大好，给人想散架的的感觉。哈哈，总体上还可以。装系统有点绕手，害的我装了一个小时。,0
1,2，散热是有点问题，CPU cache 是小了点。不能运行很大的软件。,0
2,上月入住，将近500元一天的房费，卫生间很小，像经济性酒店。走廊房间一股霉味。浴缸下水管漏水...,0
3,CTRIP上怎么让它这么忽悠顾客的 ？！！！！！！！,0
4,于丹教授讲《论语》不能很正确地反映儒家原来的思想，她除了讲《论语》不好外，别的还可以。,0


#### 分词：

选择一种多进程的 Apply 方式来分词

In [5]:
def _apply_df(args):
    df, func, num, kwargs = args
    return num, df.apply(func, **kwargs)

def apply_by_multiprocessing(df,func,**kwargs):
    workers=kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, i, kwargs) for i,d in enumerate(np.array_split(df, workers))])
    pool.close()
    result=sorted(result,key=lambda x:x[0])
    return pd.concat([i[1] for i in result])

In [6]:
df['cut_words'] = apply_by_multiprocessing(df.review, jieba.lcut, workers=4)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.004 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.022 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.069 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.104 seconds.
Prefix dict has been built succesfully.


In [7]:
df.tail()

,review,sentiment,cut_words
35119,很多人一直强调研究红楼，我其实真的有点搞不懂为什么一定要研究到底不可。就像刘心武说秦可卿是太...,1?,"[很多, 人, 一直, 强调, 研究, 红楼, ，, 我, 其实, 真的, 有点, 搞不懂,..."
35120,“主管会计必读”这本书独具匠心，一改革以往这类版本书的编排风格，从内容上编排上更加贴近实际工...,1?,"[“, 主管, 会计, 必读, ”, 这, 本书, 独具匠心, ，, 一, 改革, 以往, ..."
35121,对明朝的历史基本上是空白的，因为办公室里的一个玩笑，开始接触这本书，越看越喜欢，眼前浮现出一...,1?,"[对, 明朝, 的, 历史, 基本上, 是, 空白, 的, ，, 因为, 办公室, 里, 的..."
35122,昨天买了这个本本，收到后一看开始搞活动送内存，送硬盘了....订单完成了也没有办法了...感...,1?,"[昨天, 买, 了, 这个, 本本, ，, 收到, 后, 一看, 开始, 搞, 活动, 送,..."
35123,顺便提一句，杭州的司机开车真是野蛮，想怎么开就怎么开，相比之下上海的私家车主自律性真的很好。,1?,"[顺便, 提, 一句, ，, 杭州, 的, 司机, 开车, 真是, 野蛮, ，, 想, 怎么..."


停止词对于情感分析毫无帮助，所以剔除

In [8]:
with open('data/stop_words_chinese.txt') as file:
    data = file.read()

In [9]:
stop_words_chinese = data.split('\n')

In [10]:
def remove_stop_words(l): return [s for s in l if s not in stop_words_chinese]

In [11]:
df['cleared_words'] = apply_by_multiprocessing(df.cut_words, remove_stop_words, workers=4)

移除中文停止词之后，发现还有英文的标点符号

In [12]:
def remove_english_punctuation(l): return [s for s in l if s not in string.punctuation ]

In [13]:
df['cleared_words'] = apply_by_multiprocessing(df.cleared_words, remove_english_punctuation, workers=4)

In [14]:
df.head()

,review,sentiment,cut_words,cleared_words
0,光驱不大好，给人想散架的的感觉。哈哈，总体上还可以。装系统有点绕手，害的我装了一个小时。,0,"[光驱, 不大好, ，, 给, 人, 想, 散架, 的, 的, 感觉, 。, 哈哈, ，, ...","[光驱, 不大好, 想, 散架, 感觉, 总体, 装, 系统, 有点, 绕手, 害, 装, ..."
1,2，散热是有点问题，CPU cache 是小了点。不能运行很大的软件。,0,"[2, ，, 散热, 是, 有点, 问题, ，, CPU, , cache, , 是, ...","[散热, 有点, 问题, CPU, cache, 点, 不能, 运行, 很大, 软件]"
2,上月入住，将近500元一天的房费，卫生间很小，像经济性酒店。走廊房间一股霉味。浴缸下水管漏水...,0,"[上, 月, 入住, ，, 将近, 500, 元, 一天, 的, 房费, ，, 卫生间, 很...","[月, 入住, 将近, 500, 元, 一天, 房费, 卫生间, 很小, 经济性, 酒店, ..."
3,CTRIP上怎么让它这么忽悠顾客的 ？！！！！！！！,0,"[CTRIP, 上, 怎么, 让, 它, 这么, 忽悠, 顾客, 的, , ？, ！, ！...","[CTRIP, 忽悠, 顾客]"
4,于丹教授讲《论语》不能很正确地反映儒家原来的思想，她除了讲《论语》不好外，别的还可以。,0,"[于, 丹, 教授, 讲, 《, 论语, 》, 不能, 很, 正确, 地, 反映, 儒家, ...","[丹, 教授, 讲, 论语, 不能, 正确, 反映, 儒家, 原来, 思想, 讲, 论语, ..."


In [15]:
df['counter'] = apply_by_multiprocessing(df.cleared_words, Counter, workers=4)

数据长度，方便计算总长度

In [16]:
df['words_count'] = df.cleared_words.map(len)

In [17]:
df.head(2)

,review,sentiment,cut_words,cleared_words,counter,words_count
0,光驱不大好，给人想散架的的感觉。哈哈，总体上还可以。装系统有点绕手，害的我装了一个小时。,0,"[光驱, 不大好, ，, 给, 人, 想, 散架, 的, 的, 感觉, 。, 哈哈, ，, ...","[光驱, 不大好, 想, 散架, 感觉, 总体, 装, 系统, 有点, 绕手, 害, 装, ...","{'绕手': 1, '一个': 1, '想': 1, '装': 2, '总体': 1, '散...",14
1,2，散热是有点问题，CPU cache 是小了点。不能运行很大的软件。,0,"[2, ，, 散热, 是, 有点, 问题, ，, CPU, , cache, , 是, ...","[散热, 有点, 问题, CPU, cache, 点, 不能, 运行, 很大, 软件]","{'cache': 1, '不能': 1, '散热': 1, '软件': 1, '很大': ...",10


In [18]:
df_train = df[(df.sentiment == 1 )| (df.sentiment ==0)]
df_train_pos =  df[df.sentiment == 1]
df_train_neg = df[df.sentiment == 0]
df_test_pos = df[df.sentiment == '1?']
df_test_neg = df[df.sentiment == '0?']

$$\hat{P}(c) = \frac{N_{c}}{N}$$
$$\hat{P}(w|c) = \frac{count(w,c)+1}{count(c)+|V|}$$

首先计算 $\hat{P}(c)$，即先验概率

In [19]:
p_pos = df_train_pos.shape[0] / df_train.shape[0]
p_neg = 1 - p_pos


print('pos 的先验概率是 {}'.format(p_pos))
print('neg 的先验概率是 {}'.format(p_neg))

pos 的先验概率是 0.47112177662084115
neg 的先验概率是 0.5288782233791589


计算 $count(c)$

In [20]:
count_pos = df_train_pos.words_count.sum()
count_neg = df_train_neg.words_count.sum()

计算 $V$

直接用 pandas 或 numpy 的 sum 函数、方法计算 Couter 总数时，速度太慢，所以还是多进程吧

In [21]:
def other_multiprocessing(df, func, workers):

    chunk_size = int(df.shape[0] / workers)
    chunks = (df.ix[df.index[i:i + chunk_size]] for i in range(0, df.shape[0], chunk_size))

    pool = multiprocessing.Pool(processes=4)
    result = pool.map(func, chunks)
    return result


def sum_func(d):
    return d.counter.sum()

In [22]:
%%time
result = other_multiprocessing(df_train, sum_func, workers=4)
train_counter_sum = np.sum(np.asarray(result))

CPU times: user 1.12 s, sys: 284 ms, total: 1.41 s
Wall time: 1min 13s


In [23]:
V = len(train_counter_sum.keys())

计算 $count(c)$

In [24]:
count_pos = df_train_pos.words_count.sum()
count_neg = df_train_neg.words_count.sum()

In [25]:
%%time
p_counter_sum_list = other_multiprocessing(df_train_pos, sum_func, workers=4)
pos_counter_sum = np.sum(np.asarray(p_counter_sum_list))

CPU times: user 532 ms, sys: 184 ms, total: 716 ms
Wall time: 22.5 s


In [26]:
%%time
n_counter_sum_list = other_multiprocessing(df_train_neg, sum_func, workers=4)
neg_counter_sum = np.sum(np.asarray(n_counter_sum_list))

CPU times: user 524 ms, sys: 136 ms, total: 660 ms
Wall time: 21.2 s


写两个验证情绪的函数

In [27]:
def compute_ppd(lst):
    l = []
    for key in lst:
        l.append((pos_counter_sum[key] + 1) / (count_pos + V + 1))
    l.append(p_pos)
    a = np.asarray(l)
    return a.prod()


def compute_pnd(lst):
    l = []
    for key in lst:
        l.append((neg_counter_sum[key] + 1) / (count_neg + V + 1))
    l.append(p_neg)
    a = np.asarray(l)
    return a.prod()

In [28]:
pd.options.mode.chained_assignment = None

In [29]:
df_test_pos['ppd'] = apply_by_multiprocessing(
    df_test_pos.cleared_words, compute_ppd, workers=4)

In [30]:
df_test_pos['pnd'] = apply_by_multiprocessing(
    df_test_pos.cleared_words, compute_pnd, workers=4)

In [31]:
df_test_neg['pnd'] = apply_by_multiprocessing(
    df_test_neg.cleared_words, compute_pnd, workers=4)

In [32]:
df_test_neg['ppd'] = apply_by_multiprocessing(
    df_test_neg.cleared_words, compute_ppd, workers=4)

In [33]:
df_test_pos['predict_sentiment'] = np.where(df_test_pos.ppd > df_test_pos.pnd, 1, 0)
df_test_neg['predict_sentiment'] = np.where(df_test_neg.ppd > df_test_neg.pnd, 1, 0)

In [34]:
df_test_pos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4965 entries, 30159 to 35123
Data columns (total 9 columns):
review               4965 non-null object
sentiment            4965 non-null object
cut_words            4965 non-null object
cleared_words        4965 non-null object
counter              4965 non-null object
words_count          4965 non-null int64
ppd                  4965 non-null float64
pnd                  4965 non-null float64
predict_sentiment    4965 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 387.9+ KB


生成混淆矩阵

名称|含义|预测方向
----|----|--------
True Negative|正确预测成负例|0 -> 0
True Positive|正确预测成正例|1 -> 1
False Positive|错误预测成正例(即负例预测成正例)|0 -> 1
False Negative|错误预测成负例(即正例预测成负例)|1 -> 0

In [35]:
actual_positive= df_test_pos.predict_sentiment.value_counts()

In [36]:
actual_negative = df_test_neg.predict_sentiment.value_counts()

In [37]:
TN = actual_negative[0]
FN = actual_positive[0]

TP = actual_positive[1]
FP = actual_negative[1]

In [38]:
confusion_matrix = pd.DataFrame(
    {'predicted_no': [TN, FN], 'predicted_yes': [FP, TP]}, 
    index=['actual_no', 'actual_yes'])
confusion_matrix

,predicted_no,predicted_yes
actual_no,4843,730
actual_yes,1161,3804


In [39]:
predicted_no = confusion_matrix['predicted_no'].sum()
predicted_yes = confusion_matrix['predicted_yes'].sum()

In [40]:
total = actual_negative.sum() + actual_positive.sum()
actual_yes = confusion_matrix.loc['actual_yes'].sum()
actual_no = confusion_matrix.loc['actual_no'].sum()
predicted_no = confusion_matrix['predicted_no'].sum()
predicted_yes = confusion_matrix['predicted_yes'].sum()

In [41]:
accuracy = (TP + TN) / total
misclassification_rate = (FP + FN)/ total
ture_positive_rate = TP / actual_yes
false_positive_rate = FP / actual_no
specificity = TN / actual_no
precision = TP / predicted_yes
prevalence = actual_yes / total

In [42]:
print("accuracy is: {:.2f}".format(accuracy))
print("misclassification_rate is: {:.2f}".format(misclassification_rate))
print("ture_positive_rate is: {:.2f}".format(ture_positive_rate))
print("false_positive_rate is: {:.2f}".format(false_positive_rate))
print("specificity is: {:.2f}".format(specificity))
print("precision is: {:.2f}".format(precision))
print("prevalence is: {:.2f}".format(prevalence))

accuracy is: 0.82
misclassification_rate is: 0.18
ture_positive_rate is: 0.77
false_positive_rate is: 0.13
specificity is: 0.87
precision is: 0.84
prevalence is: 0.47
